# CIFAR10 with MLPs
Simple starter notebook to benchmark your own MLP with PyTorch on the CIFAR-10 dataset.

OBS.:

- The main code is basically done, so focus on training the models and searching for the best hyperparameters and architectures.
- You are not required to use this exact code or even the PyTorch library.
- It is recommended to use execution environments with GPU access (such as Google Colab), since larger models will take more time to train.
- Remember to document the history of your experiments and which results motivated the changes in subsequent experiments.

In [ ]:
#@title Libs

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

from tqdm import tqdm

In [ ]:
#@title Dataset Setup

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


100%|██████████| 170M/170M [00:03<00:00, 43.7MB/s]


In [ ]:
#@title Defining the MLP model
# 3072 (input) → 64 → 128 → 64 → 10 (output)

class MLP(nn.Module):
    def __init__(self, input_size=3072, num_classes=10):
        super(MLP, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [ ]:
#@title Defining metrics helper

def get_scores(targets, predictions):
    return {
        "accuracy": metrics.accuracy_score(targets, predictions),
        "balanced_accuracy": metrics.balanced_accuracy_score(targets, predictions),
        "precision": metrics.precision_score(targets, predictions, average="weighted"),
        "recall": metrics.recall_score(targets, predictions, average="weighted"),
        "f1_score": metrics.f1_score(targets, predictions, average="weighted")
    }

In [ ]:
#@title Hyperparameters
input_size = 32*32*3 # 32x32 RGB images
num_classes = 10

learning_rate = 0.001
num_epochs = 20
batch_size = 128

loss_function = nn.CrossEntropyLoss()

In [ ]:
#@title Loaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#@title Training loop

# Build the model
mlp = MLP(input_size=input_size, num_classes=num_classes)
mlp.cuda()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Early stopping setup
best_loss = float('inf')
patience = 5
patience_counter = 0

# Start training epochs loop
for epoch in tqdm(range(num_epochs)):
  mlp.train()
  epoch_loss = 0.0
  correct = 0
  total = 0

  for i, (images, labels) in enumerate(train_loader):
    images = images.view(-1,32*32*3).cuda() # flattenning images
    labels = labels.cuda()

    # Forward pass
    optimizer.zero_grad()
    outputs = mlp(images)
    loss = criterion(outputs, labels)

    # Backward pass
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

    # Acurácia
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    if (i+1) % 1000 == 0:
      tqdm.write(f' Epoch {epoch + 1}/{num_epochs}, Step {i+1}/{len(train_dataset) // batch_size}, Loss: {loss}')

  epoch_loss /= len(train_loader)
  train_accuracy = 100 * correct / total
  tqdm.write(f'Epoch {epoch+1} average loss: {epoch_loss:.4f}, Accuracy: {train_accuracy:.2f}%')

  # Early stopping using loss value
  if epoch_loss < best_loss:
    best_loss = epoch_loss
    patience_counter = 0
  else:
    patience_counter += 1
    if patience_counter >= patience:
      tqdm.write("Early stopping triggered.")
      break

  5%|▌         | 1/20 [00:14<04:44, 14.99s/it]

Epoch 1 average loss: 1.7215, Accuracy: 38.91%


 10%|█         | 2/20 [00:29<04:24, 14.67s/it]

Epoch 2 average loss: 1.5522, Accuracy: 45.20%


 15%|█▌        | 3/20 [00:43<04:03, 14.33s/it]

...

Epoch 18 average loss: 1.0125, Accuracy: 63.85%


 95%|█████████▌| 19/20 [04:27<00:14, 14.01s/it]

Epoch 19 average loss: 0.9946, Accuracy: 64.25%


100%|██████████| 20/20 [04:41<00:00, 14.09s/it]

Epoch 20 average loss: 0.9727, Accuracy: 65.11%


In [ ]:
#@title Evaluate model (accuracy, precision, recall)
mlp.eval()
predictions = []
labels = []
for images, label in test_loader:
  images = images.view(-1,32*32*3).cuda()
  label = label.cuda()

  output = mlp(images)
  _, predicted = torch.max(output,1)

  predictions.extend(predicted.cpu().numpy())
  labels.extend(label.cpu().numpy())

scores = get_scores(labels, predictions)
print("Scores of your model\n", scores)

Scores of your model
 {'accuracy': 0.557, 'balanced_accuracy': np.float64(0.5569999999999999), 'precision': 0.5674116721417998, 'recall': 0.557, 'f1_score': 0.557585513800575}


# You can change/optimize this as you want
- Different optimizers, activation functions, etc
- Automatic hyperparameters optimization (Optuna)
- Regularization techniques
- Validation set to track metrics during epochs
- Transform input data
- ...